In [21]:
import pandas as pd
from tqdm import tqdm

# Get entities

In [1]:
from kg_tools.wikitools import get_entities_of_category

In [2]:
BASE_CAT_URL = 'https://hu.wikipedia.org/w/index.php?title=Kateg%C3%B3ria:Magyar_k%C3%B6lt%C5%91k&from=A'

In [ ]:
entity_df = get_entities_of_category(BASE_CAT_URL)
entity_df

Vitéz Ferenc failed.
Bék Timur failed.
Halmi Tibor failed.


In [ ]:
entity_df['wikidata_id'] = entity_df.wikidata_url.apply(lambda s: s.split('/')[-1])

In [ ]:
entity_df

In [ ]:
entity_df.to_json('data/poet_entites.json', orient='records')

# Get triplets

In [ ]:
from kg_tools.kg_creation import triplet_for_id

In [ ]:
entity_df = pd.read_json('data/poet_entites.json', orient='records')

In [ ]:
poet_ids = entity_df.wikidata_id.unique().tolist()

In [ ]:
triplets = []

for _id in tqdm(poet_ids):
    triplets.append(triplet_for_id(_id))
    
triplets_df = pd.concat(triplets)

In [ ]:
triplets_df.to_json('data/poet_triplets.json', orient='records')

# Query edge properties

In [14]:
entity_df

,name,wiki_url,wikidata_url,wikidata_id
0,Bencze József (költő),/wiki/Bencze_J%C3%B3zsef_(k%C3%B6lt%C5%91),https://www.wikidata.org/wiki/Special:EntityPa...,Q17361654
1,Erg Ágoston,/wiki/Erg_%C3%81goston,https://www.wikidata.org/wiki/Special:EntityPa...,Q990595
2,Szokolay Zoltán,/wiki/Szokolay_Zolt%C3%A1n,https://www.wikidata.org/wiki/Special:EntityPa...,Q1299686
3,Payer Imre (költő),/wiki/Payer_Imre_(k%C3%B6lt%C5%91),https://www.wikidata.org/wiki/Special:EntityPa...,Q1231706
4,Nógrádi Gábor,/wiki/N%C3%B3gr%C3%A1di_G%C3%A1bor,https://www.wikidata.org/wiki/Special:EntityPa...,Q1221507
...,...,...,...,...
2049,Fekete Tivadar,/wiki/Fekete_Tivadar,https://www.wikidata.org/wiki/Special:EntityPa...,Q998206
2050,Erődi Dániel József,/wiki/Er%C5%91di_D%C3%A1niel_J%C3%B3zsef,https://www.wikidata.org/wiki/Special:EntityPa...,Q991159
2051,Balázs F. Attila,/wiki/Bal%C3%A1zs_F._Attila,https://www.wikidata.org/wiki/Special:EntityPa...,Q789844
2052,Babik József,/wiki/Babik_J%C3%B3zsef,https://www.wikidata.org/wiki/Special:EntityPa...,Q787889


In [15]:
entity_df.to_json('data/poet_entites.json', orient='records')

# Get triplets

In [16]:
from kg_tools.kg_creation import triplet_for_id

In [17]:
entity_df = pd.read_json('data/poet_entites.json', orient='records')

In [20]:
poet_ids = entity_df.wikidata_id.unique().tolist()

In [22]:
triplets = []

for _id in tqdm(poet_ids):
    triplets.append(triplet_for_id(_id))
    
triplets_df = pd.concat(triplets)

100%|██████████| 2053/2053 [15:31<00:00,  2.20it/s]


In [23]:
triplets_df.to_json('data/poet_triplets.json', orient='records')

# Query edge properties